## Yale New Haven Preprocessing - LSA
### Balanced

In [ ]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import TruncatedSVD

Load the data

In [ ]:
training_feats_filepath = "~/scratch/datasets/yale_new_haven/training_test_sets/balanced_dataset/features/normalized_preprocessing/regression_nn/yale_new_haven_balanced_training_features.csv"
test_feats_filepath = "~/scratch/datasets/yale_new_haven/training_test_sets/balanced_dataset/features/normalized_preprocessing/regression_nn/yale_new_haven_balanced_test_features.csv"

In [ ]:
X_train = pd.read_csv(training_feats_filepath)
X_test = pd.read_csv(test_feats_filepath)

In [ ]:
disposition_var = {'disposition'}
demographic_vars = {'age', 'gender', 'ethnicity', 'race', 'lang',
       'religion', 'maritalstatus', 'employstatus', 'insurance_status'}
triage_evaluation_vars = {'dep_name', 'esi', 'arrivalmode', 'arrivalmonth', 'arrivalday', 'arrivalhour_bin'}.union({col for col in X_train.columns if 'triage_vital' in col})
chief_complaint_vars = {col for col in X_train.columns if "cc_" in col}
medication_vars = {col for col in X_train.columns if 'meds_' in col}
hospital_usage_stats_vars = {'previousdispo', 'n_edvisits', 'n_admissions', 'n_surgeries'}
imaging_ekg_vars = {'cxr_count','echo_count','ekg_count','otherxr_count', 'otherus_count', 'headct_count', 'otherct_count', 'mri_count','otherimg_count'}
historical_vital_vars = {'dbp_last',
 'dbp_max',
 'dbp_median',
 'dbp_min',
 'o2_device_last',
 'o2_device_max',
 'o2_device_median',
 'o2_device_min',
 'pulse_last',
 'pulse_max',
 'pulse_median',
 'pulse_min',
 'resp_last',
 'resp_max',
 'resp_median',
 'resp_min',
 'sbp_last',
 'sbp_max',
 'sbp_median',
 'sbp_min',
 'spo2_last',
 'spo2_max',
 'spo2_median',
 'spo2_min',
 'temp_last',
 'temp_max',
 'temp_median',
 'temp_min'}
curr = disposition_var.union(demographic_vars.union(triage_evaluation_vars.union(chief_complaint_vars.union(medication_vars.union(hospital_usage_stats_vars.union(imaging_ekg_vars.union(historical_vital_vars)))))))
past_medical_hist_vars = {col for col in X_train.columns if col not in curr and "_" not in col and col not in ['ID', 'previousdispo']}

cc_cols = list(chief_complaint_vars)
pmh_cols = list(past_medical_hist_vars)

cc_cols.sort()
pmh_cols.sort()

LSA on the Chief Complaint and the Past Medical History columns

In [ ]:
tfidf_cc = TfidfTransformer()
tfidf_pmh = TfidfTransformer()

In [ ]:
cc_long_features = tfidf_cc.fit_transform(X_train[cc_cols])
pmh_long_features = tfidf_pmh.fit_transform(X_train[pmh_cols])

Fit SVD

In [ ]:
# truncated is recommended for sparse matrices like TF-IDF
k = 50
tsvd_cc = TruncatedSVD(k)
tsvd_pmh = TruncatedSVD(k)

cc_train_features = tsvd_cc.fit_transform(cc_long_features)
pmh_train_features = tsvd_pmh.fit_transform(pmh_long_features)

Transform the training and test sets

In [ ]:
# convert to df
cc_train_features_df = pd.DataFrame(cc_train_features, columns=[f"cc_{i}" for i in range(cc_train_features.shape[1])])
pmh_train_features_df = pd.DataFrame(pmh_train_features, columns=[f"pmh_{i}" for i in range(pmh_train_features.shape[1])])

# get other features
other_train_features = X_train[[col for col in X_train.columns if col not in cc_cols and col not in pmh_cols]]

# combine results
X_train = pd.concat([other_train_features, cc_train_features_df, pmh_train_features_df], axis=1)

In [ ]:
# apply transformation
cc_test_features = tsvd_cc.transform(tfidf_cc.transform(X_test[cc_cols]))
pmh_test_features = tsvd_pmh.transform(tfidf_pmh.transform(X_test[pmh_cols]))

# convert to df
cc_test_features_df = pd.DataFrame(cc_test_features, columns=[f"cc_{i}" for i in range(cc_test_features.shape[1])])
pmh_test_features_df = pd.DataFrame(pmh_test_features, columns=[f"pmh_{i}" for i in range(pmh_test_features.shape[1])])

other_test_features = X_test[[col for col in X_test.columns if col not in cc_cols and col not in pmh_cols]]

X_test = pd.concat([other_test_features, cc_test_features_df, pmh_test_features_df], axis=1)

Save the results

In [ ]:
X_train.to_csv("~/scratch/datasets/yale_new_haven/training_test_sets/balanced_dataset/features/LSA/lsa_balanced_training_features.csv", index=False)
X_test.to_csv("~/scratch/datasets/yale_new_haven/training_test_sets/balanced_dataset/features/LSA/lsa_balanced_test_features.csv", index=False)